# Introdução

Este notebook tem o intuito de relacionar as bases 1 e 2 conforme descritos abaixo:
    
1. "Emission_Country_Year" - Base principal do projeto, pois contén os dados quantitativos relacionados a emissão de gases poluentes por País e Ano, esta base registra dados de AAAA a AAAA de 236 países.

2. "Growth population" - Base auxiliar, contém informações sobre o Crescimento populacional por País,

As relações das bases se darão pelas colunas Base 1, Coluna "country" e Base 2, coluna "Entity" que são comuns em ambas as bases.

Os dados foram limitados para dar inicio no ano de 1850, por conta da nossa base principal iniciar neste ano.

Contextualização: Utilizaremos está base para dar mais detalhes de como a Emissão de cO2 foi influenciada com o crescimento populacional de cada País. 

# Configurações e importações de bibliotecas

In [1]:
!pip install google-cloud-storage
!pip install pandas

In [2]:
# Importar a biblioteca do 
import pandas as pd

# Importar a biblioteca do Google Cloud Storage para upload do arquivo .csv salvo
from google.cloud import storage

# Criação da Dimensão País 

In [3]:
# Ler a base dimensão País do Github
Pais = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv', error_bad_lines=False)

# Listar todas as colunas para visualizar quais serão utilizadas
Pais.columns

Index(['iso_code', 'country', 'year', 'co2', 'co2_growth_prct',
       'co2_growth_abs', 'consumption_co2', 'trade_co2', 'trade_co2_share',
       'co2_per_capita', 'consumption_co2_per_capita', 'share_global_co2',
       'cumulative_co2', 'share_global_cumulative_co2', 'co2_per_gdp',
       'consumption_co2_per_gdp', 'co2_per_unit_energy', 'cement_co2',
       'coal_co2', 'flaring_co2', 'gas_co2', 'oil_co2', 'other_industry_co2',
       'cement_co2_per_capita', 'coal_co2_per_capita',
       'flaring_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita',
       'other_co2_per_capita', 'share_global_coal_co2', 'share_global_oil_co2',
       'share_global_gas_co2', 'share_global_flaring_co2',
       'share_global_cement_co2', 'cumulative_coal_co2', 'cumulative_oil_co2',
       'cumulative_gas_co2', 'cumulative_flaring_co2', 'cumulative_cement_co2',
       'share_global_cumulative_coal_co2', 'share_global_cumulative_oil_co2',
       'share_global_cumulative_gas_co2',
       'share_g

In [5]:
# Selecionar as colunas que serão utilizadas
Pais = Pais.filter(items=['iso_code', 'country'])

# Dropar todos os valores nulos
#Pais.dropna(inplace=True)

In [6]:
#Agrupar 'iso_code' selecionando os dados coluna 'country' - índice reiniciado para 'iso_code' e transformado em coluna
Pais = Pais.groupby('country')['iso_code'].apply(lambda Pais: Pais.reset_index(drop=True)).unstack(0)

#Dropar todos os valores nulos
Pais.dropna(inplace=True)

# Transpor DataFrame
Pais = Pais.transpose()

# Reiniciar índice
Pais.reset_index(inplace=True)

# Alterar nome da coluna para 'country'
Pais.columns = [['country', 'iso_code']]

#exibição do dataframe
Pais.head()

,country,iso_code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,Andorra,AND
4,Angola,AGO


In [6]:
# Salvar o dataframe em formato .csv na Máquina Virtual
Pais.to_csv('/home/jupyter/Growth_Pais.csv')

In [7]:
# Especificar chave de acesso ao Google Cloud Storage
chavegcs = storage.Client.from_service_account_json(json_credentials_path='/home/projeto/projeto-integrado-testes-d05ad511204a.json')

# Nome do Bucket onde o arquivo será salvo no Google Cloud Storage
bucket = chavegcs.get_bucket('projeto-integrado')

# Nome que o arquivo será salvo no Google Cloud Storage
nomearquivogcs = bucket.blob('Growth_Pais.csv')

# Diretório/Nome do arquivo na VM a ser salvo no Google Cloud Storage
nomearquivogcs.upload_from_filename('/home/jupyter/Growth_Pais.csv')

# Tratamento de Crescimento populacional por País

In [7]:
# Ler a base do PIB do diretorio do GitHub
GrowthPais  = pd.read_csv('https://raw.githubusercontent.com/ViniGome/Projeto-Integrado/main/population.csv', error_bad_lines=False)

# Listar todas as colunas para visualizar quais serão utilizadas
GrowthPais .columns

Index(['Entity', 'Code', 'Year', 'Total population (Gapminder, HYDE & UN)'], dtype='object')

In [8]:
# Selecionar as colunas que serão utilizadas
GrowthPais = GrowthPais .filter(items=['Year', 'Entity', 'Total population (Gapminder, HYDE & UN)'])

In [9]:
# Dropando dados abaixo do ano de 1850.
indexNames = GrowthPais[(GrowthPais['Year'] < 1850 )].index
GrowthPais.drop(indexNames, inplace=True)
GrowthPais

,Year,Entity,"Total population (Gapminder, HYDE & UN)"
50,1850,Afghanistan,3810047
51,1851,Afghanistan,3826140
52,1852,Afghanistan,3842299
53,1853,Afghanistan,3858524
54,1854,Afghanistan,3874815
...,...,...,...
46878,2015,Zimbabwe,13815000
46879,2016,Zimbabwe,14030000
46880,2017,Zimbabwe,14237000
46881,2018,Zimbabwe,14439000


In [10]:
# Converter coluna dt para número inteiro
GrowthPais = GrowthPais .astype({'Year': int})

# Alterar nomes das colunas
GrowthPais = GrowthPais .rename(columns={'Entity': 'Country', 'Total population (Gapminder, HYDE & UN)': 'Growth'})

In [14]:
#importação de pacote
from google.cloud import storage

In [15]:
# setando as credeciais com o arquivo json que contem a chave
client = storage.Client.from_service_account_json(json_credentials_path='/home/greyce_cos_sil/projetoteste-299919-abc73be2d5b5.json')

In [16]:
# setando o bucket onde deseja-se salvar os arquivos
bucket = client.get_bucket('jupyter_note')

In [17]:
#salvando os dados do dataframe Kyoto_Agreement_Countries em um arquivo csv
GrowthPais.to_csv("Growth_Country_Year.csv")

In [18]:
#Nome do arquivo que será salvo no bucket
Growth_Country_Year_bucket = bucket.blob('Growth_Country_Year.csv')

In [20]:
# Name of the object in local file system
Growth_Country_Year_bucket.upload_from_filename('/home/jupyter/Growth_Country_Year.csv')

In [21]:
# Calcular as médias do PIB por Ano e País.
MediaGrowthPais = GrowthPais.groupby(['Year', 'Country']).mean()

# Visualizar o novo dataframe
MediaGrowthPais.head()

Growth
Year Country               
1850 Afghanistan    3810047
     Africa       102932088
     Albania         506889
     Algeria        3299305
     Andorra           3230

In [22]:
# Salvar o dataframe em formato .csv na Máquina Virtual
MediaGrowthPais.to_csv('/home/jupyter/Growth-Country.csv')

In [23]:
# Nome que o arquivo será salvo no Google Cloud Storage
nomearquivogcs3 = bucket.blob('Growth-Country.csv')

# Diretório/Nome do arquivo na VM a ser salvo no Google Cloud Storage
nomearquivogcs3.upload_from_filename('/home/jupyter/Growth-Country.csv')

Autor do notebook >>>>>>>>>>>>>>>>>>> Vinicius